# Baseline vs Fine-tuned YOLOv11s Comparison

## Overview

Notebook này so sánh performance giữa:
- **Baseline**: YOLOv11s pretrained (không fine-tune)
- **Fine-tuned**: YOLOv11s đã fine-tune trên chest X-ray data

**Các bước thực hiện:**
1. Load cả 2 models
2. Evaluate trên test set
3. Tạo comparison plots:
   - mAP comparison
   - Precision/Recall comparison
   - Per-class performance
   - Confusion matrices
4. Lưu plots cho báo cáo

**Thời gian hoàn thành:** ~15-20 phút

---

## Section 1: Setup and Imports

In [1]:
# Set working directory to repository root
%cd /home/minhquana/workspace/project_DeepLearning/computer_vision/Abnormal-Prediction-In-Chest-X-Ray

/home/minhquana/workspace/project_DeepLearning/computer_vision/Abnormal-Prediction-In-Chest-X-Ray


In [2]:
# Import required libraries
import os
from pathlib import Path
import yaml
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch

from ultralytics import YOLO

# Set style for plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Imports successful")
print(f"  PyTorch version: {torch.__version__}")
print(f"  CUDA available: {torch.cuda.is_available()}")

✓ Imports successful
  PyTorch version: 2.9.0+cu128
  CUDA available: True


## Section 2: Load Models and Data

In [3]:
# Model paths
baseline_model_path = 'models/yolo11s.pt'  # Pretrained model
finetuned_model_path = Path('models/best.pt')

# Data configuration
data_yaml_path = Path('data/preprocessed/data.yaml')

print("Loading Models and Data Configuration")
print("=" * 80)

# Check if fine-tuned model exists
if not finetuned_model_path.exists():
    print(f"ERROR: Fine-tuned model not found at {finetuned_model_path}")
    print("\nPlease run train_yolov11s.ipynb first to train the model.")
    raise FileNotFoundError(f"Fine-tuned model not found")

# Check if data.yaml exists
if not data_yaml_path.exists():
    print(f"ERROR: data.yaml not found at {data_yaml_path}")
    print("\nPlease run data_preparation.ipynb first to create preprocessed data.")
    raise FileNotFoundError(f"data.yaml not found")

print(f"✓ Baseline model: {baseline_model_path}")
print(f"✓ Fine-tuned model: {finetuned_model_path}")
print(f"✓ Data config: {data_yaml_path}")

# Load data configuration
with open(data_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

class_names = data_config['names']
num_classes = data_config['nc']

print(f"\nDataset Info:")
print(f"  Number of classes: {num_classes}")
print(f"  Class names: {class_names}")

print("=" * 80)

Loading Models and Data Configuration
✓ Baseline model: models/yolo11s.pt
✓ Fine-tuned model: models/best.pt
✓ Data config: data/preprocessed/data.yaml

Dataset Info:
  Number of classes: 2
  Class names: ['Aortic enlargement', 'Cardiomegaly']


In [4]:
# Load models
print("\nLoading models...")

print("  Loading baseline model...")
baseline_model = YOLO(baseline_model_path)
print("    ✓ Baseline model loaded")

print("  Loading fine-tuned model...")
finetuned_model = YOLO(str(finetuned_model_path))
print("    ✓ Fine-tuned model loaded")

print("\n✓ Both models loaded successfully")


Loading models...
  Loading baseline model...
    ✓ Baseline model loaded
  Loading fine-tuned model...
    ✓ Fine-tuned model loaded

✓ Both models loaded successfully


## Section 3: Evaluate Models on Test Set

In [5]:
print("\nEvaluating Baseline Model on Test Set")
print("=" * 80)

baseline_metrics = baseline_model.val(
    data=str(data_yaml_path),
    split='test',
    save_json=True,
    save_hybrid=True,
    conf=0.25,
    iou=0.6,
    max_det=300,
    plots=True,
)

baseline_results = baseline_metrics.results_dict

print("\nBaseline Results:")
print(f"  mAP50:       {baseline_results.get('metrics/mAP50(B)', 0):.4f}")
print(f"  mAP50-95:    {baseline_results.get('metrics/mAP50-95(B)', 0):.4f}")
print(f"  Precision:   {baseline_results.get('metrics/precision(B)', 0):.4f}")
print(f"  Recall:      {baseline_results.get('metrics/recall(B)', 0):.4f}")
print("=" * 80)


Evaluating Baseline Model on Test Set
WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in the future.
Ultralytics 8.3.226 🚀 Python-3.12.3 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 11906MiB)
YOLO11s summary (fused): 100 layers, 9,443,760 parameters, 0 gradients, 21.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4606.9±1455.3 MB/s, size: 77.4 KB)
val: Scanning /home/minhquana/workspace/project_DeepLearning/computer_vision/Abnormal-Prediction-In-Chest-X-Ray/data/preprocessed/test/labels... 745 images, 322 backgrounds, 95 corrupt: 100% ━━━━━━━━━━━━ 745/745 5.2Kit/s 0.1s0.1s
val: /home/minhquana/workspace/project_DeepLearning/computer_vision/Abnormal-Prediction-In-Chest-X-Ray/data/preprocessed/test/images/013893a5fa90241c65c3efcdbdd2cec1_png.rf.e47e67421a60244c1d52476d97b88d69.jpg: ignoring corrupt image/label: Label class 6 exceeds dataset class count 2. Possible class labels are 0-1
val: /home/minhquana/workspace/project_DeepLearning/computer_vision/Abnorma

In [6]:
print("\nEvaluating Fine-tuned Model on Test Set")
print("=" * 80)

finetuned_metrics = finetuned_model.val(
    data=str(data_yaml_path),
    split='test',
    save_json=True,
    save_hybrid=True,
    conf=0.25,
    iou=0.6,
    max_det=300,
    plots=True,
)

finetuned_results = finetuned_metrics.results_dict

print("\nFine-tuned Results:")
print(f"  mAP50:       {finetuned_results.get('metrics/mAP50(B)', 0):.4f}")
print(f"  mAP50-95:    {finetuned_results.get('metrics/mAP50-95(B)', 0):.4f}")
print(f"  Precision:   {finetuned_results.get('metrics/precision(B)', 0):.4f}")
print(f"  Recall:      {finetuned_results.get('metrics/recall(B)', 0):.4f}")
print("=" * 80)


Evaluating Fine-tuned Model on Test Set
WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in the future.
Ultralytics 8.3.226 🚀 Python-3.12.3 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 11906MiB)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4010.0±1620.3 MB/s, size: 79.6 KB)
val: Scanning /home/minhquana/workspace/project_DeepLearning/computer_vision/Abnormal-Prediction-In-Chest-X-Ray/data/preprocessed/test/labels.cache... 745 images, 322 backgrounds, 95 corrupt: 100% ━━━━━━━━━━━━ 745/745 2.0Mit/s 0.0s0s
val: /home/minhquana/workspace/project_DeepLearning/computer_vision/Abnormal-Prediction-In-Chest-X-Ray/data/preprocessed/test/images/013893a5fa90241c65c3efcdbdd2cec1_png.rf.e47e67421a60244c1d52476d97b88d69.jpg: ignoring corrupt image/label: Label class 6 exceeds dataset class count 2. Possible class labels are 0-1
val: /home/minhquana/workspace/project_DeepLearning/computer_vision/A

## Section 4: Create Comparison Plots

In [7]:
# Create plots directory
plots_dir = Path('docs/comparison_plots')
plots_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

print(f"Plots will be saved to: {plots_dir}")

Plots will be saved to: docs/comparison_plots


In [8]:
# Plot 1: Overall Metrics Comparison
print("\nCreating Overall Metrics Comparison...")

metrics_to_compare = [
    ('mAP50', 'metrics/mAP50(B)'),
    ('mAP50-95', 'metrics/mAP50-95(B)'),
    ('Precision', 'metrics/precision(B)'),
    ('Recall', 'metrics/recall(B)'),
]

baseline_values = [baseline_results.get(key, 0) for _, key in metrics_to_compare]
finetuned_values = [finetuned_results.get(key, 0) for _, key in metrics_to_compare]
metric_names = [name for name, _ in metrics_to_compare]

x = np.arange(len(metric_names))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width/2, baseline_values, width, label='Baseline', alpha=0.8, color='#FF6B6B')
bars2 = ax.bar(x + width/2, finetuned_values, width, label='Fine-tuned', alpha=0.8, color='#4ECDC4')

ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('Baseline vs Fine-tuned YOLOv11s - Overall Metrics Comparison', fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(metric_names, fontsize=11)
ax.legend(fontsize=11, loc='upper right')
ax.set_ylim([0, 1.0])
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Add value labels on bars
def add_value_labels(bars):
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=10, fontweight='bold')

add_value_labels(bars1)
add_value_labels(bars2)

plt.tight_layout()
plot_path = plots_dir / f'overall_metrics_comparison_{timestamp}.png'
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f"  ✓ Saved to: {plot_path}")
plt.show()


Creating Overall Metrics Comparison...
  ✓ Saved to: docs/comparison_plots/overall_metrics_comparison_20251111_040806.png


<Figure size 1200x600 with 1 Axes>

In [9]:
# Plot 2: Improvement Percentage
print("\nCreating Improvement Percentage Plot...")

improvements = []
for baseline_val, finetuned_val in zip(baseline_values, finetuned_values):
    if baseline_val > 0:
        improvement = ((finetuned_val - baseline_val) / baseline_val) * 100
    else:
        improvement = 0
    improvements.append(improvement)

fig, ax = plt.subplots(figsize=(12, 6))
colors = ['#2ECC71' if imp >= 0 else '#E74C3C' for imp in improvements]
bars = ax.bar(metric_names, improvements, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)

ax.set_ylabel('Improvement (%)', fontsize=12, fontweight='bold')
ax.set_title('Performance Improvement: Fine-tuned vs Baseline', fontsize=14, fontweight='bold', pad=20)
ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Add value labels
for bar, imp in zip(bars, improvements):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{imp:+.1f}%',
            ha='center', va='bottom' if imp >= 0 else 'top',
            fontsize=11, fontweight='bold')

plt.tight_layout()
plot_path = plots_dir / f'improvement_percentage_{timestamp}.png'
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f"  ✓ Saved to: {plot_path}")
plt.show()


Creating Improvement Percentage Plot...
  ✓ Saved to: docs/comparison_plots/improvement_percentage_20251111_040806.png


<Figure size 1200x600 with 1 Axes>

In [10]:
# Plot 3: Per-Class mAP50 Comparison (if available)
print("\nCreating Per-Class mAP50 Comparison...")

# Try to get per-class AP from metrics
if hasattr(baseline_metrics, 'box') and hasattr(baseline_metrics.box, 'ap_class_index'):
    baseline_ap_per_class = baseline_metrics.box.ap50
    finetuned_ap_per_class = finetuned_metrics.box.ap50
    
    # Create comparison DataFrame
    comparison_df = pd.DataFrame({
        'Class': class_names[:len(baseline_ap_per_class)],
        'Baseline': baseline_ap_per_class,
        'Fine-tuned': finetuned_ap_per_class,
    })
    
    # Sort by improvement
    comparison_df['Improvement'] = comparison_df['Fine-tuned'] - comparison_df['Baseline']
    comparison_df = comparison_df.sort_values('Improvement', ascending=False)
    
    fig, ax = plt.subplots(figsize=(14, max(8, len(comparison_df) * 0.5)))
    
    y_pos = np.arange(len(comparison_df))
    width = 0.35
    
    bars1 = ax.barh(y_pos - width/2, comparison_df['Baseline'], width, label='Baseline', alpha=0.8, color='#FF6B6B')
    bars2 = ax.barh(y_pos + width/2, comparison_df['Fine-tuned'], width, label='Fine-tuned', alpha=0.8, color='#4ECDC4')
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels(comparison_df['Class'], fontsize=10)
    ax.set_xlabel('mAP50', fontsize=12, fontweight='bold')
    ax.set_title('Per-Class mAP50 Comparison', fontsize=14, fontweight='bold', pad=20)
    ax.legend(fontsize=11, loc='lower right')
    ax.set_xlim([0, 1.0])
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plot_path = plots_dir / f'per_class_map50_{timestamp}.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"  ✓ Saved to: {plot_path}")
    plt.show()
    
    # Display table
    print("\n📋 Per-Class mAP50 Table:")
    print(comparison_df.to_string(index=False))
else:
    print("  Per-class AP not available in metrics")


Creating Per-Class mAP50 Comparison...
  ✓ Saved to: docs/comparison_plots/per_class_map50_20251111_040806.png


<Figure size 1400x800 with 1 Axes>


📋 Per-Class mAP50 Table:
             Class  Baseline  Fine-tuned  Improvement
      Cardiomegaly       0.0    0.908275     0.908275
Aortic enlargement       0.0    0.872789     0.872789


In [11]:
# Plot 4: Model Size and Speed Comparison
print("\n⚡ Creating Model Size and Speed Comparison...")

# Get model info
baseline_size = Path(baseline_model_path).stat().st_size / (1024 * 1024)  # MB
finetuned_size = finetuned_model_path.stat().st_size / (1024 * 1024)  # MB

# Get inference speed (if available from metrics)
baseline_speed = baseline_results.get('speed/inference', 0) if 'speed/inference' in baseline_results else 0
finetuned_speed = finetuned_results.get('speed/inference', 0) if 'speed/inference' in finetuned_results else 0

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Model size comparison
models = ['Baseline', 'Fine-tuned']
sizes = [baseline_size, finetuned_size]
colors = ['#FF6B6B', '#4ECDC4']

bars1 = ax1.bar(models, sizes, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax1.set_ylabel('Model Size (MB)', fontsize=12, fontweight='bold')
ax1.set_title('Model Size Comparison', fontsize=13, fontweight='bold')
ax1.grid(axis='y', alpha=0.3, linestyle='--')

for bar, size in zip(bars1, sizes):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{size:.1f} MB',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

# Inference speed comparison (if available)
if baseline_speed > 0 and finetuned_speed > 0:
    speeds = [baseline_speed, finetuned_speed]
    bars2 = ax2.bar(models, speeds, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    ax2.set_ylabel('Inference Speed (ms)', fontsize=12, fontweight='bold')
    ax2.set_title('Inference Speed Comparison', fontsize=13, fontweight='bold')
    ax2.grid(axis='y', alpha=0.3, linestyle='--')
    
    for bar, speed in zip(bars2, speeds):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                 f'{speed:.2f} ms',
                 ha='center', va='bottom', fontsize=11, fontweight='bold')
else:
    ax2.text(0.5, 0.5, 'Speed data not available', 
             ha='center', va='center', fontsize=12, transform=ax2.transAxes)
    ax2.set_title('Inference Speed Comparison', fontsize=13, fontweight='bold')

plt.tight_layout()
plot_path = plots_dir / f'model_size_speed_{timestamp}.png'
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f"  ✓ Saved to: {plot_path}")
plt.show()


⚡ Creating Model Size and Speed Comparison...
  ✓ Saved to: docs/comparison_plots/model_size_speed_20251111_040806.png


<Figure size 1400x500 with 2 Axes>

## Section 5: Generate Summary Report

In [12]:
# Create summary report
print("\nGenerating Summary Report...")

report_lines = [
    "# Baseline vs Fine-tuned YOLOv11s - Comparison Report\n",
    f"**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n",
    "## Models\n",
    f"- **Baseline**: YOLOv11s pretrained (`{baseline_model_path}`)\n",
    f"- **Fine-tuned**: YOLOv11s fine-tuned on chest X-ray data (`{finetuned_model_path}`)\n\n",
    "## Dataset\n",
    f"- **Test set**: `{data_yaml_path}`\n",
    f"- **Number of classes**: {num_classes}\n",
    f"- **Classes**: {', '.join(class_names)}\n\n",
    "## Overall Performance Comparison\n\n",
    "| Metric | Baseline | Fine-tuned | Improvement |\n",
    "|--------|----------|------------|-------------|\n",
]

for (name, _), baseline_val, finetuned_val, improvement in zip(
    metrics_to_compare, baseline_values, finetuned_values, improvements
):
    report_lines.append(
        f"| {name} | {baseline_val:.4f} | {finetuned_val:.4f} | {improvement:+.2f}% |\n"
    )

report_lines.extend([
    "\n## Key Findings\n\n",
])

# Add key findings based on improvements
avg_improvement = np.mean(improvements)
if avg_improvement > 0:
    report_lines.append(f"**Overall improvement**: {avg_improvement:.2f}% average increase across metrics\n\n")
else:
    report_lines.append(f"**Overall performance**: {avg_improvement:.2f}% change (needs investigation)\n\n")

# Find best and worst performing metrics
best_metric_idx = np.argmax(improvements)
worst_metric_idx = np.argmin(improvements)

report_lines.extend([
    f"- **Best improvement**: {metric_names[best_metric_idx]} ({improvements[best_metric_idx]:+.2f}%)\n",
    f"- **Least improvement**: {metric_names[worst_metric_idx]} ({improvements[worst_metric_idx]:+.2f}%)\n\n",
])

report_lines.extend([
    "## Model Properties\n\n",
    f"- **Baseline size**: {baseline_size:.2f} MB\n",
    f"- **Fine-tuned size**: {finetuned_size:.2f} MB\n",
])

if baseline_speed > 0 and finetuned_speed > 0:
    report_lines.extend([
        f"- **Baseline inference**: {baseline_speed:.2f} ms\n",
        f"- **Fine-tuned inference**: {finetuned_speed:.2f} ms\n",
    ])

report_lines.extend([
    "\n## Visualizations\n\n",
    f"All comparison plots saved to: `{plots_dir}/`\n\n",
    "- Overall metrics comparison\n",
    "- Improvement percentage\n",
    "- Per-class mAP50 (if available)\n",
    "- Model size and speed\n\n",
    "## Conclusion\n\n",
])

if avg_improvement > 5:
    report_lines.append(
        "The fine-tuned model shows **significant improvement** over the baseline, "
        "indicating successful domain adaptation to chest X-ray abnormality detection. "
        "The model is ready for production deployment.\n"
    )
elif avg_improvement > 0:
    report_lines.append(
        "The fine-tuned model shows **modest improvement** over the baseline. "
        "Consider additional training epochs or data augmentation strategies.\n"
    )
else:
    report_lines.append(
        "The fine-tuned model shows **no improvement** or degradation. "
        "Review training configuration, data quality, and augmentation strategies.\n"
    )

# Save report
report_path = plots_dir / f'comparison_report_{timestamp}.md'
with open(report_path, 'w', encoding='utf-8') as f:
    f.writelines(report_lines)

print(f"  ✓ Report saved to: {report_path}")

# Display report
print("\n" + "=" * 80)
print("COMPARISON REPORT")
print("=" * 80)
print(''.join(report_lines))
print("=" * 80)


Generating Summary Report...
  ✓ Report saved to: docs/comparison_plots/comparison_report_20251111_040806.md

COMPARISON REPORT
# Baseline vs Fine-tuned YOLOv11s - Comparison Report
**Generated:** 2025-11-11 04:08:07

## Models
- **Baseline**: YOLOv11s pretrained (`models/yolo11s.pt`)
- **Fine-tuned**: YOLOv11s fine-tuned on chest X-ray data (`models/best.pt`)

## Dataset
- **Test set**: `data/preprocessed/data.yaml`
- **Number of classes**: 2
- **Classes**: Aortic enlargement, Cardiomegaly

## Overall Performance Comparison

| Metric | Baseline | Fine-tuned | Improvement |
|--------|----------|------------|-------------|
| mAP50 | 0.0000 | 0.8905 | +0.00% |
| mAP50-95 | 0.0000 | 0.4986 | +0.00% |
| Precision | 0.0000 | 0.8532 | +0.00% |
| Recall | 0.0000 | 0.8841 | +0.00% |

## Key Findings

**Overall performance**: 0.00% change (needs investigation)

- **Best improvement**: mAP50 (+0.00%)
- **Least improvement**: mAP50 (+0.00%)

## Model Properties

- **Baseline size**: 18.42 MB
- *

## Section 6: Summary

In [13]:
print("\nCOMPARISON COMPLETED")
print("=" * 80)

print("\nGenerated Assets:")
print(f"  Plots directory: {plots_dir}/")
print(f"  Summary report: {report_path}")

print("\nQuick Summary:")
print(f"  Average improvement: {avg_improvement:+.2f}%")
print(f"  Best metric: {metric_names[best_metric_idx]} ({improvements[best_metric_idx]:+.2f}%)")
print(f"  Baseline mAP50: {baseline_results.get('metrics/mAP50(B)', 0):.4f}")
print(f"  Fine-tuned mAP50: {finetuned_results.get('metrics/mAP50(B)', 0):.4f}")

print("\nNext Steps:")
print("  1. Review generated plots in docs/comparison_plots/")
print("  2. Include plots in your report/presentation")
print("  3. Analyze per-class performance for weak classes")
print("  4. Deploy fine-tuned model if performance is satisfactory")

print("\n" + "=" * 80)


COMPARISON COMPLETED

Generated Assets:
  Plots directory: docs/comparison_plots/
  Summary report: docs/comparison_plots/comparison_report_20251111_040806.md

Quick Summary:
  Average improvement: +0.00%
  Best metric: mAP50 (+0.00%)
  Baseline mAP50: 0.0000
  Fine-tuned mAP50: 0.8905

Next Steps:
  1. Review generated plots in docs/comparison_plots/
  2. Include plots in your report/presentation
  3. Analyze per-class performance for weak classes
  4. Deploy fine-tuned model if performance is satisfactory

